In [ ]:
import numpy as np
import scipy as sp
import pandas as pd 
from matplotlib import pyplot as plt
import matplotlib
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import acf, pacf
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import ParameterGrid
from joblib import Parallel, delayed
from tqdm.notebook import tqdm
from datetime import timedelta
import sys
import os
import utils

for item in ['../utils', '../portal']:
    module_path = os.path.abspath(item)
    if module_path not in sys.path:
        sys.path.append(module_path)
    
plt.rcParams['figure.figsize'] = [16, 4]

path_data = "../data"
path_portal = "../portal"
year = 2023

# HCKT03 - Time Series Forecasting

## 1. Data Cleaning
- Impute missing values using interpolation or statistical methods.
- Remove invalid PM2.5 values exceeding 165 µg/m³.
- Aggregate data spatially across sites to address gaps and inconsistencies.

## 2. Data Exploration
- Visualize PM2.5 and NOx readings for each site. Highlight trends, outliers, and missing values.
- Analyze daily and hourly patterns for both variables to identify recurring peaks or anomalies.
- Examine cross-correlations between PM2.5 and NOx to uncover lagged and lead relationships.
- Apply seasonal decomposition to separate trend, seasonal, and residual components in the PM2.5 data.
- Compare PM2.5 patterns between early November and December to determine if levels are improving or worsening as the year progresses.


## 3. Feature Engineering
- Create lagged features for PM2.5 and NOx to capture temporal dependencies (e.g., 1-hour, 6-hour, and 1-day lags).
- Add time-based features, such as hour of the day and day of the week, to model seasonal trends.
- Spatially aggregate NOx as a single exogenous variable for modeling.
- Test the predictive power of real NOx data versus forecasted NOx.

## 4. Modeling
- **Baseline Models**: Build Linear Regression and Gradient Boosting models incorporating NOx as an exogenous variable.
- **Improved Models**: Test SARIMAX models with NOx as an exogenous variable. Perform stationarity checks (e.g., ADF test) and apply differencing if needed.
- **Advanced Models**: Experiment with Random Forest and XGBoost for multivariate regression, combining PM2.5 and NOx features.
- Evaluate model performance using metrics such as Mean Absolute Error (MAE), Root Mean Squared Error (RMSE), and Akaike Information Criterion (AIC).

## 1. Data Cleaning

### Fetch dataset

In [ ]:
df_ref = pd.read_csv(f'{path_data}/train.csv')
df_ref

In [ ]:
df = df_ref.copy()
df.datetime = pd.to_datetime(df.datetime)
df = df.set_index(['datetime','site_id'], drop=True).sort_index()

sites = df.index.get_level_values('site_id').unique()
sensors = df.columns
df

In [ ]:
df_exog = pd.read_csv(f'{path_data}/nox_forecast.csv')
df_exog.datetime = pd.to_datetime(df_exog.datetime)
df_exog = df_exog.set_index('datetime', drop=True).sort_index()
df_exog

### Handle extreme values

Valid values for `pm25` values range from `0` to `165`.

In [ ]:
df.describe().loc[['min','max']]

In [ ]:
upper_threshold = 165
selected = df[df.pm25 > upper_threshold]
idx_selected = selected.index
iloc_selected = [df.index.get_loc(idx) for idx in idx_selected]

for idx in iloc_selected:
    df.iloc[idx, df.columns.get_loc('pm25')] = df.iloc[idx-1, df.columns.get_loc('pm25')]
    
df.describe().loc[['min','max']]

## 2. Data Exploration

### Generate surrogate data for daily analysis

In [ ]:
df_spatial = df.groupby('datetime').mean()
df_spatial = df_spatial.resample('h').mean()
df_spatial = df_spatial.interpolate(method='linear')
df_spatial = df_spatial.dropna()

df_exog = df_exog.resample('h').mean()

df_spatial_d = df_spatial.resample('d').mean()

### Visualize Dataset

- **Seasonal Decomposition**: Reveals expected weekly and seasonal patterns in the data.  
- **Splitting the Dataset**: Splitting the data between mid-October and early November ensures stationarity within each segment.

In [ ]:
sensor = 'pm25'
data = df_spatial[sensor]
decomposition = seasonal_decompose(data, model='additive')

fig = decomposition.plot()
fig.axes[0].set_ylabel('Original')
fig.axes[3].lines[0].set_markersize(3)
fig.axes[3].set_xlabel('Time')
plt.show()

In [ ]:
data = df_spatial_d[sensor]
decomposition_d = seasonal_decompose(data, model='additive')

fig = decomposition_d.plot()
fig.axes[0].set_ylabel('Original')
fig.axes[3].lines[0].set_markersize(3)
fig.axes[3].set_xlabel('Time')
plt.show()

### Winter summer split to regain stationarity
After a few tries, we settle for splitting on 1st November.

In [ ]:
day = '2023-11-01'

df_spatial_summer = df_spatial.loc[:day]
df_spatial_winter = df_spatial.loc[day:]

df_spatial_d_summer = df_spatial_d.loc[:day]
df_spatial_d_winter = df_spatial_d.loc[day:]

In [ ]:
sensor = 'pm25'
data = df_spatial_d_summer[sensor]
decomposition_d_summer = seasonal_decompose(data, model='additive')

fig = decomposition_d_summer.plot()
fig.axes[0].set_ylabel('Original')
fig.axes[3].lines[0].set_markersize(3)
fig.axes[3].set_xlabel('Time')
plt.show()

In [ ]:
sensor = 'pm25'
data = df_spatial_d_winter[sensor]
decomposition_d_winter = seasonal_decompose(data, model='additive')

fig = decomposition_d_winter.plot()
fig.axes[0].set_ylabel('Original')
fig.axes[3].lines[0].set_markersize(3)
fig.axes[3].set_xlabel('Time')
plt.show()

### Out of Curiosity - Residual Distribution Plots

To visually estimate the normality of residuals, plot histograms of residuals:

- **Daily Data**: Insufficient data points for a reliable estimation.  
- **Hourly Data**: Residuals suggest normality, but as a **multiplicative process** — applying a logarithmic transformation (**logged data**) may improve modeling results.

In [ ]:
sensor = 'pm25'
resid_d = decomposition_d.resid
resid_d_summer = decomposition_d_summer.resid
resid_d_winter = decomposition_d_winter.resid

resid_d_n = (resid_d-resid_d.mean())/resid_d.std()
resid_d_summer_n = (resid_d_summer-resid_d_summer.mean())/resid_d_summer.std()
resid_d_winter_n = (resid_d_winter-resid_d_winter.mean())/resid_d_winter.std()

bins = np.linspace(-3,3, num=16)
plt.hist(resid_d_n, bins=bins, density=True, alpha=0.5, label='all')
plt.hist(resid_d_summer_n, bins=bins, density=True, alpha=0.5, label='summer')
plt.hist(resid_d_winter_n, bins=bins, density=True, alpha=0.5, label='winter')
plt.title(f'Histogram of residuals {sensor}')
plt.legend()
plt.show()

In [ ]:
sensor = 'pm25'
resid = seasonal_decompose(df_spatial[sensor], model='additive').resid
resid_summer = seasonal_decompose(df_spatial_summer[sensor], model='additive').resid
resid_winter = seasonal_decompose(df_spatial_winter[sensor], model='additive').resid

resid_n = (resid-resid.mean())/resid.std()
resid_summer_n = (resid_summer-resid_summer.mean())/resid_summer.std()
resid_winter_n = (resid_winter-resid_winter.mean())/resid_winter.std()

x = np.linspace(-3, 3, num=1000)
y = sp.stats.norm.pdf(x)

bins = np.linspace(-3,3, num=26)

plt.hist(resid_n, bins=bins, alpha=0.3, density=True, label='all')
plt.hist(resid_summer_n, bins=bins, alpha=0.3, density=True, label='summer')
plt.hist(resid_winter_n, bins=bins, alpha=0.3, density=True, label='winter')
plt.plot(x, y, color='red', linewidth=2)
plt.title(f'Histogram of residuals (additive) {sensor}')
plt.legend()
plt.show()

In [ ]:
eps = 1e-6
sensor = 'pm25'
resid = seasonal_decompose(df_spatial[sensor]+eps, model='multiplicative').resid
resid_summer = seasonal_decompose(df_spatial_summer[sensor]+eps, model='multiplicative').resid
resid_winter = seasonal_decompose(df_spatial_winter[sensor]+eps, model='multiplicative').resid

resid_n = (resid-resid.mean())/resid.std()
resid_summer_n = (resid_summer-resid_summer.mean())/resid_summer.std()
resid_winter_n = (resid_winter-resid_winter.mean())/resid_winter.std()

x = np.linspace(-3, 3, num=1000)
y = sp.stats.norm.pdf(x)

bins = np.linspace(-3,3, num=26)

plt.hist(resid_n, bins=bins, alpha=0.3, density=True, label='all')
plt.hist(resid_summer_n, bins=bins, alpha=0.3, density=True, label='summer')
plt.hist(resid_winter_n, bins=bins, alpha=0.3, density=True, label='winter')
plt.plot(x, y, color='red', linewidth=2)
plt.title(f'Histogram of residuals (multiplicative) {sensor}')
plt.legend()
plt.show()

### Plot partial autocorrelation

In [ ]:
def pacf_top(data, n=5):
    values = pacf(data)
    idx_sort = np.argsort(np.abs(values))[::-1]
    for idx in idx_sort[1:n+1]:
        print(f'{idx}\t{values[idx]}')

In [ ]:
sensor = 'pm25'
lags = range(1, 48)
data = df_spatial[sensor]

plot_pacf(data, lags=lags)
plt.xlabel('Lag (h)')
plt.ylabel('Partial Autocorrelation')
plt.title(f'Partial Autocorrelation of {sensor}')
plt.ylim([-0.25,1])
plt.show()

pacf_top(data)

In [ ]:
sensor = 'pm25'
lags = range(1, 48)
data = df_spatial_summer[sensor]

plot_pacf(data, lags=lags)
plt.xlabel('Lag (h)')
plt.ylabel('Partial Autocorrelation')
plt.title(f'Partial Autocorrelation of {sensor}  - Summer')
plt.ylim([-0.25,1])
plt.show()

pacf_top(data)

In [ ]:
sensor = 'pm25'
lags = range(1, 48)
data = df_spatial_winter[sensor]

plot_pacf(data, lags=lags)
plt.xlabel('Lag (h)')
plt.ylabel('Partial Autocorrelation')
plt.title(f'Partial Autocorrelation of {sensor} - Winter')
plt.ylim([-0.25,1])
plt.show()

pacf_top(data)

## 3. Feature Engineering

### Regaining Stationarity

In [ ]:
my_df = df_spatial_winter

df_train_val = my_df.iloc[:-24*10]
df_test = my_df.iloc[-24*10:]
df_train = df_train_val.iloc[:-24*10]
df_val = df_train_val.iloc[-24*10:]

print(df_train_val.index.min(), df_train_val.index.max())
print(df_train.index.min(), df_train.index.max())
print(df_val.index.min(), df_val.index.max())
print(df_test.index.min(), df_test.index.max())

In [ ]:
sensor = 'pm25'
lags = range(1, 48)
data = df_train[sensor]

plot_pacf(data, lags=lags)
plt.xlabel('Lag (h)')
plt.ylabel('Partial Autocorrelation')
plt.title(f'Partial Autocorrelation of {sensor} - Winter')
plt.ylim([-0.25,1])
plt.show()

pacf_top(data)

## 4. Modeling

In [ ]:
test = df_test.pm25
train_val = df_train_val.pm25
train = df_train.pm25
val = df_val.pm25

In [ ]:
param_grid = {
    'model': [LinearRegression(), GradientBoostingRegressor(n_estimators=20, random_state=10)], 
    'num_lags': [0,1,2,3],
    'num_diffs': [0,1,2],
    'weekday':[True],
    'month':[False],
    'holidays': [False],
    'rolling' : [[]],
}

grid = ParameterGrid(param_grid)

In [ ]:
%%time 

def wrap_model_selection(params): 
    predictions = utils.multistep_forecast(
        original=train, 
        model=params['model'], 
        num_lags=params['num_lags'],
        num_diffs=params['num_diffs'],
        weekday=params['weekday'],
        month=params['month'],
        rolling=params['rolling'],
        holidays=params['holidays'],
        num_periods_ahead=len(val), 
    )
    return params,mean_absolute_error(val, predictions)

grid_search_result = Parallel(n_jobs=-1)(
    delayed(wrap_model_selection)(params=params) 
    for params in tqdm(grid)
)

df_cv = pd.DataFrame(grid_search_result, columns=['params','mae']).sort_values('mae')

best_params = df_cv.iloc[0].params
best_params

### Baseline model

In [ ]:
my_params = best_params
forecast_multi_step = utils.multistep_forecast(
     original=train_val,
     model=my_params['model'], 
     num_lags=my_params['num_lags'],
     num_diffs=my_params['num_diffs'],
     weekday=my_params['weekday'],
     month=my_params['month'],
     rolling=my_params['rolling'],
     holidays=my_params['holidays'],
     num_periods_ahead=len(test)
)

mae_multi_step = mean_absolute_error(test, forecast_multi_step)
mae_multi_step

In [ ]:
plt.plot(test, label='original')
plt.plot(pd.Series(forecast_multi_step,index=test.index), label='multi-step forecast')
plt.xlabel('Time')
plt.ylabel(sensor)
plt.legend();

### Introducing `exog` forecast - same timestep signal

In [ ]:
exog = df_exog.exog

In [ ]:
%%time

param_grid_exog = dict(param_grid)
param_grid_exog['exog'] = [exog]
grid_exog = ParameterGrid(param_grid_exog)

def wrap_model_selection(params): 
    predictions = utils.multistep_forecast(
        original=train, 
        model=params['model'], 
        num_lags=params['num_lags'],
        num_diffs=params['num_diffs'],
        weekday=params['weekday'],
        month=params['month'],
        rolling=params['rolling'],
        holidays=params['holidays'],
        num_periods_ahead=len(val),
        exog=params['exog'],
    )
    return params, mean_absolute_error(val, predictions)

grid_search_result_exog = Parallel(n_jobs=-1)(
    delayed(wrap_model_selection)(params=params)
    for params in tqdm(grid_exog)
)

df_cv_exog = pd.DataFrame(grid_search_result_exog, columns=['params','mae']).sort_values('mae')

best_params_exog = df_cv_exog.iloc[0].params

{key:value for key, value in best_params_exog.items() if key!='exog'}

In [ ]:
my_params = best_params_exog

forecast_multi_step_exog = utils.multistep_forecast(
     original=train_val,
     model=my_params['model'], 
     num_lags=my_params['num_lags'],
     num_diffs=my_params['num_diffs'],
     weekday=my_params['weekday'],
     month=my_params['month'],
     rolling=my_params['rolling'],
     holidays=my_params['holidays'],
     num_periods_ahead=len(test),
     exog=my_params['exog'],
)

mae_multi_step = mean_absolute_error(test, forecast_multi_step_exog)
mae_multi_step

In [ ]:
plt.plot(pd.Series(forecast_multi_step_exog, index=test.index), label='multi-step forecast')
plt.plot(test, label='original')
plt.xlabel('Time')
plt.ylabel(sensor)
plt.legend()
plt.show()

### What has the model learned vs exog signal

In [ ]:
data = pd.Series(forecast_multi_step_exog, index=test.index)
data = (data-data.mean())/data.std()
data_ref = exog.loc[data.index]
data_ref = (data_ref-data_ref.mean())/data_ref.std()

plt.plot(data, label='multi-step forecast')
plt.plot(data_ref, label='exog')
plt.xlabel('Time')
plt.ylabel(sensor)
plt.legend()
plt.show()

### Introducing `exog leads` - multiple steps into the future

In [ ]:
my_params = dict(best_params_exog)
my_params['num_lags'] = 24
my_params['num_diffs'] = 24
my_params['num_exog_leads'] = len(test)

forecast_multi_step_exog = utils.multistep_forecast(
    original=train_val,
    model=my_params['model'], 
    num_lags=my_params['num_lags'],
    num_diffs=my_params['num_diffs'],
    weekday=my_params['weekday'],
    month=my_params['month'],
    rolling=my_params['rolling'],
    holidays=my_params['holidays'],
    num_periods_ahead=len(test),
    exog=my_params['exog'],
    num_exog_leads=my_params['num_exog_leads'],
)

mae_multi_step = mean_absolute_error(test, forecast_multi_step_exog)
mae_multi_step

In [ ]:
plt.plot(pd.Series(forecast_multi_step_exog, index=test.index), label='multi-step forecast')
plt.plot(test, label='original')
plt.xlabel('Time')
plt.ylabel(sensor)
plt.legend()
plt.show()

### What has the model learned vs exog signal

In [ ]:
data = pd.Series(forecast_multi_step_exog, index=test.index)
data = (data-data.mean())/data.std()
data_ref = exog.loc[data.index]
data_ref = (data_ref-data_ref.mean())/data_ref.std()

plt.plot(data, label='multi-step forecast')
plt.plot(data_ref, label='exog')
plt.xlabel('Time')
plt.ylabel(sensor)
plt.legend()
plt.show()